In [4]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from itertools import combinations
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
data = pd.read_csv('blood_test.txt')
data2=pd.read_csv('blood_test.txt')
data2['number'] = np.arange(data.shape[0])
data

mcv  alkphos  sgpt  sgot  gammagt  drinks  selector
0     85       92    45    27       31     0.0         1
1     85       64    59    32       23     0.0         2
2     86       54    33    16       54     0.0         2
3     91       78    34    24       36     0.0         2
4     87       70    12    28       10     0.0         2
..   ...      ...   ...   ...      ...     ...       ...
340   99       75    26    24       41    12.0         1
341   96       69    53    43      203    12.0         2
342   98       77    55    35       89    15.0         1
343   91       68    27    26       14    16.0         1
344   98       99    57    45       65    20.0         1

[345 rows x 7 columns]

In [5]:
labels = data['selector'].to_numpy()
data.drop(columns='selector', inplace=True)
train_indices = random.sample(range(0, len(data)), int(0.8 * len(data)))
test_indices  = list(set(range(0, len(data))) - set(train_indices))
df_np = data.to_numpy()
train_x = df_np[train_indices]
train_y = labels[train_indices]
test_x  = df_np[test_indices]
test_y  = labels[test_indices]
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)
print('test_x shape:', test_x.shape)
print('test_y shape:', test_y.shape)

train_x shape: (276, 6)
train_y shape: (276,)
test_x shape: (69, 6)
test_y shape: (69,)


In [6]:
x_train=pd.DataFrame (train_x, columns = ['mcv', 'alkphos','sgpt','sgot','gammagt','drinks'])
y_train=pd.Series(train_y,name='selector')

In [7]:
result = pd.concat([x_train, y_train], axis=1)

In [8]:
labels1 = result['selector'].to_numpy()
result.drop(columns='selector', inplace=True)
#100 sample
train_indices1 = random.sample(range(0, len(result)), 100)
test_indices1  = list(set(range(0, len(result))) - set(train_indices1))
df_np = result.to_numpy()
train_x1 = df_np[train_indices1]
train_y1 = labels1[train_indices1]
x_train_1=pd.DataFrame (train_x1, columns = ['mcv', 'alkphos','sgpt','sgot','gammagt','drinks'])
y_train_1=pd.Series(train_y1,name='selector')

In [6]:
def fit(X_train, y_train, classes, priors, means):
  x, y = X_train.values, y_train.values
  for i, label in enumerate(classes):
    priors[i] = y[y == label].size / y.size
    means[i] = np.mean(x[y == label], axis=0)
  covariance_matrix = covariance(x, y, classes, means)
  return means, priors, covariance_matrix

#Covariance calculation function
def covariance(x, y, classes, means):
  covs = [0 for i in range(len(classes))]
  cov = np.zeros(shape=(x.shape[1], x.shape[1]))
  for i, label in enumerate(classes):
    members = x[y == label]
    x_mu = members - means[i]
    cov += (x_mu).T.dot(x_mu)
    cov /= x.shape[0] #delete if single
    covs[i] = cov # cov /= X.shape[0] out of loop if single
  return covs # return cov if single

#Probability calculation function
def probability(x, mean, prior, covariance_matrix):
  xm = x - mean
  xm_covariance = (xm.dot(np.linalg.pinv(covariance_matrix))) * xm
  xm_covariance_sum = xm_covariance.sum(axis=1)
  return -0.5 * xm_covariance_sum -(0.5) * np.log(np.linalg.det(covariance_matrix))+ np.log(prior)

#Function to obtain the result
def predict(X_train, classes, means, priors, covariance_matrix):
  x = X_train.values
  probabilities = np.zeros((x.shape[0], priors.size))
  for i,_ in enumerate(classes):
    probabilities[:, i] = probability(
        x, means[i]
        , priors[i]
        , covariance_matrix[i]) # covariance_matrix -> if single
  return np.argmax(probabilities, axis=1)

#The result probability calculation function
def predict_proba(x):
  probabilities = np.zeros((x.shape[0], priors.size))
  for i,_ in enumerate(classes):
    probabilities[:, i] = probability(
        x, means[i]
        , priors[i]
        , covariance_matrix)
  return probabilities

In [7]:
def observe(X_train,y_train):
    classes = np.unique(y_train)
    classes.sort()
    means = np.zeros((classes.size, X_train.shape[1]))
    priors = np.zeros(classes.size)
    means, priors, covariance_matrix = fit(X_train, y_train, classes, priors, means)
    predicted_ys = predict(X_train, classes, means, priors, covariance_matrix)+1
    #print(predicted_ys,y_train.tolist())

    from sklearn.metrics import accuracy_score
    #print("Train Accuracy : ", accuracy_score(y_train, predicted_ys))
    return accuracy_score(y_train, predicted_ys),predicted_ys,means,priors,covariance_matrix,classes



In [9]:
A=list(combinations(x_train,2))
Accuracy_list=[]
for i in range(len(A)):
    x1=A[i][0]
    x2=A[i][1]
    X_new_train=x_train_1[[x1,x2]]
    acc,predicted_ys,means,priors,covariance_matrix,classes=observe(X_new_train,y_train_1)
    Accuracy_list.append(acc)


ind = np.argmax(Accuracy_list)
X_new_train=x_train[[A[ind][0],A[ind][1]]]
X_new_train.head()

sgpt  sgot
0  47.0  23.0
1  13.0  17.0
2  30.0  18.0
3  41.0  20.0
4  14.0  16.0